La voiture : étude d'un fait social

Par Alexandre Pakin et Gilles Massoulié

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Introduction

L’usage de la voiture constitue un objet central d’analyse en sociologie, tant il structure les mobilités quotidiennes, les trajectoires sociales et les rapports aux territoires en France. Loin d’être un simple outil de déplacement, l’automobile peut être appréhendée comme un fait social, révélateur d’inégalités de classe, de genre, d’âge et de territoire.

Cette étude a été motivée par une [Vidéo de vulgarisation sociologique sur l’automobile](https://www.youtube.com/watch?v=tR6aujaUh_g)
 proposant une synthèse critique de plusieurs travaux récents sur l’automobile et ses usages sociaux. En mettant en évidence la dimension profondément inégalitaire de la conduite et de la possession d’un véhicule, cette vidéo invite à revenir aux données empiriques afin de reproduire et discuter certaines conclusions à partir de sources ouvertes. Le présent projet s’inscrit ainsi dans une démarche de réanalyse quantitative, visant à confronter des hypothèses sociologiques établies aux données disponibles.

La littérature récente permet de dégager plusieurs axes majeurs. D’une part, des travaux généraux de sociologie de l’automobile, comme [Sociologie de l’automobile](https://www.cairn.info/sociologie-de-l-automobile--9782707196433.htm) de Jean-Pierre Orfeuil et Frédéric Héran (2022) ou [Idées reçues sur l’automobile](https://www.cairn.info/idees-recues-sur-l-automobile--9782715405666.htm) de Frédéric Héran (2021), proposent un cadre théorique permettant de déconstruire les représentations dominantes associées à la voiture. D’autre part, plusieurs études mettent en évidence des usages différenciés selon les caractéristiques sociales des individus. Les dimensions d’âge et de genre sont notamment abordées dans des travaux portant sur la jeunesse automobile ou sur les inégalités et comportements genrés au volant, tandis que d’autres recherches analysent l’automobile comme un marqueur de position sociale et un révélateur des inégalités de classe, notamment à travers les politiques de mobilité et de sécurité routière.

Dans le prolongement de ces travaux, ce projet vise à analyser empiriquement l’usage de la voiture en France à partir de données issues de trois API publiques (data.gouv.fr, INSEE et transport.data.gouv.fr). L’objectif est d’identifier des régularités statistiques et des inégalités sociales liées à l’automobile, tout en illustrant l’apport des outils de programmation en Python pour l’analyse quantitative de phénomènes sociaux.

Sommaire (à faire)

Installation

In [ ]:
!pip install -r requirements.txt

# modules 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import re


# Fonctions
from urllib import request
from modules import module_BAAC
from modules import module_transport
from modules import graph_baac

Préparation des données

1. Adresses

Toutes les données utilisées dans ce projet sont récupérées directement en ligne, principalement via des interfaces de programmation (API) publiques. Ces API permettent un accès automatisé et reproductible aux données, ce qui s’inscrit pleinement dans une démarche de science des données appliquée aux sciences sociales.

In [ ]:
# API Bases de données annuelles des accidents corporels de la circulation routière – 2005 à 2024 (gouvernement.fr)
BAAC_url = "https://www.data.gouv.fr/api/1/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/"

# URLs d'export CSV : consommation quotidienne d’alcool (Odissé - Santé Publique France)
URL_CONSO = "https://odisse.santepubliquefrance.fr/explore/dataset/alcool-consommation-quotidienne-region/download/?format=csv"

# URLs d'export CSV : alcoolisation ponctuelle importante (Odissé - Santé Publique France)
URL_API   = "https://odisse.santepubliquefrance.fr/explore/dataset/alcool-alcoolisation-ponctuelle-importante-api-mensuel-et-hebdomadaire-region/download/?format=csv"

# URLs d'export .XLSX : impoosition par région (gouvernement.fr)
url_impot = "https://www.impots.gouv.fr/node/25517"   # page with the file

2. Liste des accidents

(à compléter, chaque ligne est un accident)

In [14]:
BAAC_resources = module_BAAC.get_resources(BAAC_url)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = 300
BAAC_resources[["description", "url"]].head(10)

,description,url
0,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2024.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100210/2024.csv
1,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2023.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142813/2023.csv
2,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv
3,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv
4,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv
5,Table des usagers impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115506/usagers-2024.csv
6,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2022.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142722/2022.csv
7,Table des usagers impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv
8,Table des véhicules impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153253/vehicules-2023.csv
9,Table des lieux des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153219/lieux-2023.csv


On a sélectionné les catégories suivantes (à rédiger), toutes les tables sauf la première qui est une liste des véhicules immatriculés dans les accidents ont en lien la variable num_acc, donc on les fusionne pour obtenir le dataframe BAAC_table.
Il faudra qd mm utiliser le premier dataframe car il est sûrement intéressant.

In [30]:
selected_BAAC_table = module_BAAC.select_baac_tables(BAAC_resources)
selected_BAAC_table.head(10)

,description,url,table,year
0,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv,caracteristiques,2024
1,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv,lieux,2024
2,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv,vehicules,2024
3,Table des usagers impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115506/usagers-2024.csv,usagers,2024
4,Table des usagers impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv,usagers,2023
5,Table des véhicules impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153253/vehicules-2023.csv,vehicules,2023
6,Table des lieux des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153219/lieux-2023.csv,lieux,2023
7,Table des caractéristiques des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241028-103125/caract-2023.csv,caracteristiques,2023
8,Table des usagers impliqués dans les accidents corporels de la circulation en 2022 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094229/usagers-2022.csv,usagers,2022
9,Table des véhicules impliqués dans les accidents corporels de la circulation en 2022 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094147/vehicules-2022.csv,vehicules,2022


In [16]:
BAAC_table = module_BAAC.build_baac_dataframe(selected_BAAC_table)
BAAC_table.head(10)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,...,dep,year,id_vehicule_x,secu1,secu2,secu3,id_vehicule_y,motor,vma,id_usager
0,200500000001,1,1,4,1,1,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200500000001,1,1,4,1,1,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200500000001,1,1,3,2,3,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200500000001,1,1,3,2,3,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200500000001,2,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,200500000001,2,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,200500000001,4,2,1,1,0,31,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,200500000001,4,2,1,1,0,31,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,200500000001,5,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,200500000001,5,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Heatmap en fonction de l'heure et du jour de la semaine

In [17]:
BAAC_table['hour'] = BAAC_table['hrmn'].str.split(':').str[0].astype(int)
BAAC_table.columns

BAAC_table.to_csv('table_test.csv')

ValueError: cannot convert float NaN to integer

In [ ]:
BAAC_table[['lat','long']].head()

,lat,long
0,"47,56277000","6,75832000"
1,"47,02109000","4,83755000"
2,"47,02109000","4,83755000"
3,"44,90238400","2,49641800"
4,"49,19166000","-0,39851000"


blabla 2e dataframe

In [ ]:
# Chargement en dataframes
df_conso = pd.read_csv(URL_CONSO, sep=';')
df_api   = pd.read_csv(URL_API, sep=';')
df_conso.head(10)

,annee,reg,reglib,sexe,alc_quo_v,alc_quo_icb,alc_quo_ich,alc_quo_typo
0,2000,11,Île-de-France,Femmes,16.2914,13.5946,19.4031,C
1,2000,24,Centre-Val de Loire,Femmes,11.0433,7.4733,16.0234,B
2,2000,28,Normandie,Femmes,8.1800,5.6390,11.7239,A
3,2000,84,Auvergne et Rhône-Alpes,Femmes,12.3394,9.9204,15.2484,B
4,2000,24,Centre-Val de Loire,Hommes,28.4894,22.4781,35.3747,B
5,2000,32,Hauts-de-France,Hommes,31.3315,26.8006,36.2490,B
6,2000,52,Pays de la Loire,Hommes,32.6807,27.1029,38.7954,B
7,2000,53,Bretagne,Hommes,31.8400,26.1168,38.1695,B
8,2000,76,Occitanie,Hommes,37.5760,32.5754,42.8563,C
9,2000,11,Île-de-France,Hommes et Femmes,22.3674,19.9049,25.0392,B


In [ ]:
response = requests.get(url_impot)
df = pd.read_excel(BytesIO(response.content))
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(nombres en milliers)
1,RÉGIONS,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023
2,ILE-DE-FRANCE,6770.6,6809.9,6831.3,6907.2,6978.2,7152.0,7241.9,7247.0,7310.0,7428
3,CENTRE VAL DE LOIRE,1448.5,1455.2,1464.0,1481.5,1476.3,1494.1,1506.0,1520.0,1531.0,1543
4,BOURGOGNE FRANCHE COMTE,1586.4,1590.8,1592.0,1598.6,1598.1,1618.9,1634.8,1643.0,1660.0,1667
5,NORMANDIE,1857.8,1865.0,1868.0,1877.3,1881.6,1912.9,1936.2,1954.0,1973.0,1987
6,HAUTS DE FRANCE,3261.9,3273.2,3278.0,3302.9,3297.6,3345.1,3383.5,3409.0,3434.0,3455
7,GRAND EST,3054.6,3065.4,3075.2,3097.7,3120.9,3155.6,3182.2,3194.0,3213.0,3234
8,PAYS DE LA LOIRE,2004.3,2026.3,2045.4,2075.8,2095.9,2143.8,2181.1,2210.0,2244.0,2269
9,BRETAGNE,1824.8,1842.4,1857.5,1884.9,1894.8,1935.9,1971.5,2006.0,2036.0,2062
